### QBIO465 Assignment 10
Due Thursday, April 24th before midnight (California time)

For this assignment, you are tasked with executing binary image segmentation on a collection of brain MRI scans using the U-Net architecture. The objective is to evaluate U-Net's segmentation accuracy in scenarios constrained by limited data and computational resources. This task will highlight U-Net's effectiveness in medical image analysis, particularly its ability to detect and outline abnormalities.

Most of the necessary code has been provided, serving as a reference for managing image projects in your final research. We will go through the code collectively, where you will be responsible for completing the sections that are missing.

#### Dataset
This dataset features brain MRI scans along with hand-crafted FLAIR abnormality segmentation masks. These images are sourced from The Cancer Imaging Archive (TCIA) and pertain to 110 patients from The Cancer Genome Atlas (TCGA) lower-grade glioma collection. Each patient's data includes at least one FLAIR sequence and associated genomic cluster information. More information: https://www.kaggle.com/datasets/mateuszbuda/lgg-mri-segmentation/data.

The dataset is featured in:  
M. Buda et al., Computers in Biology and Medicine 2019  
M. A. Mazurowski et al., Journal of Neuro-Oncology 2017

#### 1. Initialization

In [ ]:
import os
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, MaxPool2D, Conv2DTranspose, concatenate

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 64
EPOCHS = 20


#### 2. Load data
To get the data, go to https://www.kaggle.com/datasets/mateuszbuda/lgg-mri-segmentation/data. Click on "Download (749 MB)" located at the top right corner of the page to download the dataset to your local computer.

In [ ]:
# Configure the path to the image files
IMAGE_PATH = 'your path\\lgg-mri-segmentation\\kaggle_3m'

# A list to store the paths of each image and its corresponding mask
paths = glob.glob(os.path.join(IMAGE_PATH, '**', '*.tif'), recursive=True)

# Display the total number of paths found and a subset of these paths
len(paths), paths[:5]

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

def create_data_frame(data, reduce_ratio=1):
    # Filter image and mask paths
    images = [path for path in data if not path.endswith('mask.tif')]
    masks = [path for path in data if path.endswith('mask.tif')]

    # Sort images and masks by their numerical order and patient ID
    images.sort(key=lambda x: (x.rsplit('_', 3)[-2], int(x.rsplit('_', 3)[-1][:-4])))
    masks.sort(key=lambda x: (x.rsplit('_', 3)[-3], int(x.rsplit('_', 3)[-2])))

    # Extract IDs from image paths
    IDs = [path.rsplit('\\', 3)[-1][:-4] for path in images]

    # Determine diagnosis based on the presence of non-zero pixels in masks
    diagnoses = [1 if np.max(Image.open(mask)) > 0 else 0 for mask in masks]

    # Construct the DataFrame
    df = pd.DataFrame({
        'ID': IDs,
        'Image': images,
        'Mask': masks,
        'Diagnosis': diagnoses
    })

    # Reduce the DataFrame size based on the reduce_ratio when lack of the computing resource
    if reduce_ratio < 1.0:
        df = df.sample(frac=reduce_ratio).reset_index(drop=True)

    return df

# Usage
df = create_data_frame(paths, reduce_ratio=0.1)  # Adjust 'reduce_ratio' as needed
df.head()


#### 3. Prepare training, validation, and testing data for modeling

**Q1. Write a function named split_train_test_val that takes a DataFrame as input and splits it into three separate DataFrames: training, validation, and testing. The training set should contain 70% of the original data, the validation set 20%, and the testing set the remaining 10%. Ensure that the data is shuffled before splitting to avoid bias. Use a fixed random_state to ensure the reproducibility of your results. Return the three DataFrames in the order: training, validation, and testing. [1pt]**

In [ ]:
def split_train_testing(df):
    # Making train, test, and validation dataframes
    # put your code here for Q1

    return train_df, val_df, test_df

# Usage
train_df, val_df, test_df = split_train_testing(df)
print(len(train_df), len(val_df), len(test_df))
train_df.head()

In [ ]:
def plot_class_distribution(train_df, val_df, test_df):
    # Counting the class distribution for every dataframe.
    labels = ['Positive', 'Negative']
    colors = ['#4a86e8', '#6aa84f']

    # Extracting counts
    counts = {
        'Training': train_df['Diagnosis'].value_counts(),
        'Validation': val_df['Diagnosis'].value_counts(),
        'Test': test_df['Diagnosis'].value_counts()
    }

    # Setting up the plot
    fig, ax = plt.subplots(figsize=(10, 6))

    # X axis locations
    x = range(len(counts))
    total_width = 0.8
    single_width = total_width / len(labels)

    # Plotting data
    for i, label in enumerate(labels):
        values = [counts[phase].get(i, 0) for phase in counts]
        bars = ax.bar(x, values, single_width, label=label, color=colors[i])
        x = [xi + single_width for xi in x]  # Shift for the next series

        # Annotating each bar
        for bar in bars:
            height = bar.get_height()
            ax.annotate('{}'.format(height),
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

    # Setting the position of bars on X axis
    ax.set_xticks([i + total_width / len(labels) - single_width for i in range(len(counts))])
    ax.set_xticklabels(list(counts.keys()))

    # Adding labels and title
    plt.ylabel('Count')
    plt.xlabel('Dataset')
    plt.title('Class Distribution across Datasets')
    plt.legend()

    # Show plot
    plt.tight_layout()
    plt.show()

# Assuming train_df, val_df, and test_df are defined and have a 'Diagnosis' column...
plot_class_distribution(train_df, val_df, test_df)

#### 4. Vitualize the loaded data

In [ ]:
def plot_images_and_masks(images, masks, predictions=None, IoU_list=None):
    num_samples = len(images)
    num_rows = 3 if predictions is not None else 2

    fig, axes = plt.subplots(num_rows, num_samples, figsize=(num_samples * 5, num_rows * 5), dpi=100)
    if num_samples == 1:  # If only one sample, axes are not an array
        axes = np.array([[axes]])
    elif num_rows == 1:  # Ensure axes is 2D if only plotting images or masks
        axes = np.array([axes])

    titles = ['Image', 'Mask', 'Prediction']
    for i in range(num_samples):
        items_to_plot = [images[i], masks[i]] + ([predictions[i]] if predictions is not None else [])
        for j, item in enumerate(items_to_plot):
            if isinstance(item, str):  # If the item is a file path
                item = Image.open(item)
            axes[j, i].imshow(item, cmap='gray')
            if j == 2 and IoU_list:  # Specific title for predictions with IoU
                title = f'{titles[j]} | IoU: {round(float(IoU_list[i]), 3)}'
            else:
                title = titles[j]
            axes[j, i].set_title(title, fontsize=15, fontweight='bold')
            axes[j, i].axis('off')

    plt.suptitle('Images, Masks, and Predictions', fontsize=20, fontweight='bold', y=1.05)
    plt.tight_layout()
    plt.show()


In [ ]:
# Show positive cases
plot_images_and_masks(train_df[train_df['Diagnosis'] == 1]['Image'].values[:5],
                      train_df[train_df['Diagnosis'] == 1]['Mask'].values[:5])

In [ ]:
# Show negative cases
plot_images_and_masks(train_df[train_df['Diagnosis'] == 0]['Image'].values[:5],
                      train_df[train_df['Diagnosis'] == 0]['Mask'].values[:5])

#### 5. Formating Data for Model Input
In this step, '.tif' images are converted into tensors, resized to appropriate dimensions (images to 256x256x3 and masks to 256x256x1), and normalized by dividing by 255. The data is then structured using tf.data.Dataset.from_tensor_slices(), shuffled, matched (images to corresponding masks), and batched. Prefetching is applied to overlap data preprocessing with model training, optimizing resource usage and speeding up the process. This ensures the data is ready and optimized for model input.

In [ ]:
# Loads and processes the image file.
def decode_and_resize_image(img_path):
    # Load image from TIFF file format.
    img = tf.io.read_file(img_path)
    with tf.io.gfile.GFile(img_path, 'rb') as f:
        img = Image.open(f)
        img = np.array(img)
    img = tf.convert_to_tensor(img, dtype=tf.float32)
    img = tf.image.resize(img, IMAGE_SIZE, preserve_aspect_ratio=True)

    # Scale image pixel values to [0, 1].
    img = img / 255.0

    return img

# Loads and processes the mask file.
def decode_and_resize_mask(mask_path):
    # Load mask from TIFF file format.
    mask = tf.io.read_file(mask_path)
    with tf.io.gfile.GFile(mask_path, 'rb') as f:
        mask = Image.open(f)
        mask = np.array(mask)
    mask = tf.convert_to_tensor(mask, dtype=tf.float32)
    mask = np.expand_dims(mask, axis=-1)
    mask = tf.image.resize(mask, IMAGE_SIZE, method='nearest', preserve_aspect_ratio=True)
    grayscale_mask = tf.reduce_mean(mask, axis=-1, keepdims=True)

    # Scale mask pixel values to [0, 1].
    grayscale_mask = grayscale_mask / 255.0

    return grayscale_mask


In [ ]:
# Formats input images and masks for model processing.
def processed_input(img, mask):
    # Ensures images and masks are correctly shaped for the model: images as (None, 256, 256, 3), masks as (None, 256, 256, 1).
    return img, mask

# Constructs a TensorFlow dataset for model training or evaluation.
def make_dataset(images, masks):
    # Creates a dataset of image and mask pairs, applying decoding and resizing.
    dataset = tf.data.Dataset.from_tensor_slices((
        list(map(decode_and_resize_image, images)),
        list(map(decode_and_resize_mask, masks))
    ))

    # Randomly shuffles the dataset to ensure model generalization.
    dataset = dataset.shuffle(BATCH_SIZE * 8)

    # Applies formatting to each dataset item to ensure compatibility with the model input.
    dataset = dataset.map(processed_input, num_parallel_calls=tf.data.AUTOTUNE)

    # Organizes data into batches and prefetches them to improve training efficiency.
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return dataset

# Prepares training and validation datasets from provided image and mask paths.
train_dataset = make_dataset(list(train_df['Image'].values), list(train_df['Mask'].values))
validation_dataset = make_dataset(list(val_df['Image'].values), list(val_df['Mask'].values))


#### 6. Build the U-Net Model

**Q2. Implement U-Net Architecture: Create a TensorFlow function named unet that implements the U-Net architecture for image segmentation. Your function should build a model to process 256x256x3 RGB images through an encoder with convolutional blocks (each containing two Conv2D layers with LeakyReLU activation and followed by MaxPool2D), a bottleneck without pooling, and a decoder with Conv2DTranspose layers for upsampling and skip connections. The model should output a single-channel segmentation mask using a Conv2D layer with a sigmoid activation. Compile the model with the Adam optimizer (learning rate 3e-4) and 'mean_squared_error' loss. Ensure you import necessary modules from TensorFlow. Your deliverable is the unet function; test it by creating a model instance and displaying its summary. [3pt]**

In [ ]:
def unet(input_size=(256, 256, 3)):
    # put your code here for Q2

    return model

# Creating the model
model = unet()

#### 7. Train the U-Net

In [ ]:
# Specifying where to store the model's best performing weights.
best_weights_path = "unet_weights.h5"

# Set up a callback to save only the best weights based on validation loss.
model_checkpoint = ModelCheckpoint(filepath=best_weights_path,
                                   monitor='val_loss',
                                   save_best_only=True,
                                   save_weights_only=True,
                                   verbose=1)

# Initialize EarlyStopping to halt training when validation loss ceases to decrease.
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Begin model training, leveraging callbacks for checkpointing and early stopping.
history = model.fit(train_dataset, validation_data=validation_dataset,
                                   epochs=EPOCHS, callbacks=[model_checkpoint, early_stop])

# Retrieve the optimal weights once training is complete.
model.load_weights(best_weights_path)


#### 8. Plot  model's performance

**Q3. Plotting Model Performance from Training History. Utilize the training history returned by model.fit to visualize the model's performance. Create plots for both the training and validation loss over each epoch. Additionally, if available, plot the accuracy metrics for both training and validation. Ensure your plots are clearly labeled with appropriate titles, axis labels, and legends. [1pt]**

In [ ]:
## put your code here for Q3

#### 8. Testing
Intersection over Union (IoU) assesses the accuracy of predicted masks against actual ones by measuring their overlap. The predictions() function predicts masks from test data, converting them into binary format based on a 0.5 threshold, and calculates IoU for each to evaluate segmentation precision. It also classifies inputs as positive or negative diagnoses from these predictions.

About IoU: https://wiki.cloudfactory.com/docs/mp-wiki/metrics/iou-intersection-over-union
.

In [ ]:
# Function to calculate Intersection over Union (IoU) for model evaluation.
def calculate_IoU(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3]) - intersection
    return (intersection + 1e-7) / (union + 1e-7)  # Added epsilon to prevent division by zero.

# Function to predict masks, evaluate them with IoU, and compile results into a DataFrame.
def evaluate_predictions(dataframe, threshold=0.5):
    image_arrays = np.array([decode_and_resize_image(path) for path in dataframe['Image']])
    true_masks = np.array([decode_and_resize_mask(path) for path in dataframe['Mask']])

    predicted_masks = model.predict(image_arrays)
    predicted_masks_thresholded = (predicted_masks > threshold).astype(float)

    IoU_scores = calculate_IoU(true_masks, predicted_masks_thresholded).numpy()
    IoU_scores = [round(score, 3) for score in IoU_scores]

    # Determine the binary diagnosis based on the presence of mask pixels
    predicted_diagnoses = [1 if np.max(mask) > 0 else 0 for mask in predicted_masks_thresholded]

    results_df = pd.DataFrame({
        'ID': dataframe['ID'],
        'Actual Diagnosis': dataframe['Diagnosis'],
        'Predicted Diagnosis': predicted_diagnoses,
        'IoU': IoU_scores
    })

    # Creating a dictionary to map each ID to its predicted mask
    predicted_masks_dict = {ID: mask for ID, mask in zip(dataframe['ID'], predicted_masks_thresholded)}

    return predicted_masks_dict, results_df

# Utilize the function to get predictions and compile results
predicted_masks, results_dataframe = evaluate_predictions(test_df)

# Display outputs
print(f"Total Predictions: {len(predicted_masks)}, Example Mask Shape: {next(iter(predicted_masks.values())).shape}")
results_dataframe.head()


In [ ]:
results_dataframe.describe()

**Q4. Extract the actual and predicted diagnoses from results_dataframe.
Use the extracted data to calculate the confusion matrix. You are encouraged to utilize scikit-learn's confusion_matrix function for this task. [2pt]**

In [ ]:
# put your code here for Q4